In [ ]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.data import AUTOTUNE
from tensorflow.keras import layers, models, utils
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input

from config import DATASETgen, save_path
from process_data import load_data

utils.set_random_seed(0)


In [ ]:
batch_size = 32
train_dataset, val_dataset, test_dataset, total_train_samples, total_val_samples, total_test_samples = load_data(DATASETgen,
                                                                                                                 target_size=(224, 224),
                                                                                                                 batch_size=batch_size,
                                                                                                                 num_classes=2)


train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), tf.argmax(y, axis=1)),
                                  num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

val_dataset = val_dataset.map(lambda x, y: (preprocess_input(x), tf.argmax(y, axis=1)),
                              num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

test_dataset = test_dataset.map(lambda x, y: (preprocess_input(x), tf.argmax(y, axis=1)),
                                num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

In [ ]:
base_model = ResNet50(weights="imagenet",
                      include_top=False,
                      input_shape=(224, 224, 3))
base_model.trainable = False

resnet = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
resnet.summary(show_trainable=True)

In [ ]:
resnet.compile(
    optimizer=keras.optimizers.SGD(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)

In [ ]:
model = 'resnet50'
mode = 'transfer-learning'
epochs = 50
batch_size = 32
date_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filepath = os.path.join(save_path, f'{model}_{mode}_{epochs}ep_{batch_size}bs_{date_time}.keras')

callbacks = [keras.callbacks.ModelCheckpoint(filepath=filepath, save_best_only=True),
             keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2),
             keras.callbacks.EarlyStopping(patience=5)
             ]

steps_per_epoch = total_train_samples // batch_size
validation_steps = total_val_samples // batch_size

history = resnet.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

np.save(filepath.replace('.keras', '_history.npy'), history.history)

In [ ]:
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
testing_steps = total_test_samples // batch_size
test_loss, test_acc = resnet.evaluate(test_dataset,
                                            steps=testing_steps)
print(f"Test loss: {test_loss:.3f}")
print(f"Test accuracy: {test_acc:.3f}")